- drop duplicates by MLS
- merge columns
- explodes amenities into individual columns
- calculate sqft when range or room dimensions (only) are given

In [166]:
import pandas as pd
import os
import numpy as np
import ast
import re
from pathlib import Path
from collections import Counter


In [167]:
import re

def get_street_address(address):
    if isinstance(address, str):
        # Step 1: Handle missing or invalid addresses
        address = address.strip()
        
        # Step 2: If there is a dash and the part after the dash seems like an apartment number, split
        if "-" in address:
            parts = address.split("-")
            # Check if the part after the dash is likely an apartment number (contains digits)
            if re.search(r'\d', parts[1]):  # If the second part contains digits, treat as apt number
                address = parts[1]  # This assumes the apartment number is the part after the dash
            else:
                address = parts[0]  # If not, we keep the first part (before the dash)

        # Step 3: Remove any content after the opening parenthesis (e.g., neighborhoods)
        address = address.split("(")[0].strip()

        # Step 4: Remove content after "Toronto" (city name) if necessary
        address = address.split("Toronto")[0].strip()

        # Step 5: Convert to lowercase
        return address.lower()
    return ''  # If it's not a valid string, return an empty string

In [168]:
def numeric_price(df, columns_list):
    """$600,000 (str) --> 600000 (float)"""
    df_copy = df.copy()
    
    for column in columns_list:
        # Remove "$" and "," then convert to numeric, invalid parsing will become NaN
        df_copy[column] = df_copy[column].str.replace("$", "", regex=False).str.replace(",", "", regex=False)
        df_copy[column] = pd.to_numeric(df_copy[column], errors='coerce')  # Convert to float, invalid entries become NaN
        
    return df_copy


In [169]:
def numeric_lot_dim(dim):
    """Converts dimension string (e.g., '122 ft, 8 in', '40 ft', '40') to a float representing feet."""
    
    if pd.isna(dim): # For the NaN values (str operations cannot operate on float-like NaN values)
        return np.nan
    if isinstance(dim, (int, float)):
        return float(dim)
    
    dim = dim.strip().lower()

    # Initialize
    feet = 0
    inches = 0

    # Both feet and inches (e.g., "122 ft, 8 in")
    feet_in_inches = re.match(r'(\d+)\s*ft.*?(\d+)\s*in', dim)
    if feet_in_inches:
        feet = int(feet_in_inches.group(1))  # feet
        inches = int(feet_in_inches.group(2))  # inches
    else:
        # Only feet is present (e.g., "40 ft" or "40")
        feet_in_feet_only = re.match(r'(\d+)\s*ft', dim)
        if feet_in_feet_only:
            feet = int(feet_in_feet_only.group(1))
        else:
            # String is just a number (e.g., "40")
            if dim.isdigit():
                feet = int(dim)

    # Convert inches to feet (12 inches = 1 foot)
    total_feet = feet + (inches / 12)
    
    return total_feet

In [170]:
# def get_realtor_house_sigma_sqft(row):
#     """For Realtor dfs.
#     Remove 'sqft'.
#     If Square footage is a range, take the average.
#     If it is a single value, leave as is."""
    
#     # Make sure to treat row as a string and remove 'sqft'
#     if isinstance(row, str):
#         row = row.replace("sqft", "")\
#                     .replace("feet²", "")\
#                     .replace("+", "")\
#                     .replace("<", "")\
#                     .replace("<", "")\
#                     .strip()  # Remove 'units' and descriptors and strip spaces

#         # Check if it's a range (contains '-')
#         if "-" in row:
#             lower, upper = row.split("-")
#             return (float(lower.strip()) + float(upper.strip())) / 2
#         else:
#             return float(row)
#     else:
        # return None

In [171]:
def get_realtor_house_sigma_sqft(row):
    """For Realtor dfs.
    Remove 'sqft'.
    If Square footage is a range, take the average.
    If it is a single value, leave as is."""

    invalid_values = ["*******************", "******************", "****************", '***************', "N/A", "unknown", "not available", "na"]

    # Check if the row contains any of the invalid patterns (case insensitive)
    if isinstance(row, str) and any(invalid_value in row.lower() for invalid_value in invalid_values):
        return None


    # Make sure to treat row as a string and remove 'sqft'
    if isinstance(row, str):
        row = row.replace("sqft", "")\
                    .replace("feet²", "")\
                    .replace("FT", "")\
                    .replace("+", "")\
                    .replace("<", "")\
                    .replace("<", "")\
                    .strip()  # Remove 'units' and descriptors and strip spaces
                    # .replace("*", "")\

        # Check if it's a range (contains '-')
        if "-" in row:
            lower, upper = row.split("-")
            return (float(lower.strip()) + float(upper.strip())) / 2
        
        if "x" in row: # Room dimensions are in meters (later on), but Size always has units of sqft on the website
            length, width = row.split("x")
            return (float(length.strip()) * float(width.strip()))
        
        else:
            return float(row)
    
    else:
        return None

In [172]:
def get_zolo_sqft(row, room_dimensions=None):
    """For Zolo dfs. 
    If Size (sq ft) is NaN, then replaces with the sqft calculated from the room dimensions.
    If Size (sq ft) is a range (i.e., contains "-"), replaces with the average of the bounds."""
    
    # If the 'Size (sq ft)' is a range (contains "-"):
    if isinstance(row, str) and '-' in row:
        lower, upper = row.split("-")
        # Calculate the average of the two bounds
        return (float(lower) + float(upper)) / 2
    
    # If 'Size (sq ft)' is NaN, calculate from room dimensions
    elif pd.isna(row) and room_dimensions:  
        try:
            # Check if 'room_dimensions' is a list-like or string that we can process
            dimensions = ast.literal_eval(room_dimensions)
            square_meters = 0

            # If room dimensions is a list of room sizes (i.e., '200x300', '100x200', etc.)
            if isinstance(dimensions, list):
                for dimension in dimensions:
                    # Only split if 'dimension' contains 'x'
                    length, width = dimension.split("x")
                    square_meters += float(length.strip()) * float(width.strip())
                    
            return square_meters*10.7639
        
        except (ValueError, SyntaxError, TypeError):
            # Handle invalid or malformed room dimensions
            return np.nan

    elif isinstance(row, str):
        # If it's a string that doesn't contain a "-", remove any "+" and return the value as float
        return float(row.replace("+", "").replace(">", "").replace("<", "").strip())
    
    else:
        # If it's a numeric value (already a number), return it
        return row


In [173]:
# Clean the string value associated with the key "size" in each dict the "Room Info" dict for each row.

def clean_size_string(size):
    """For House Sigma dfs"""
    # Remove all non-numeric characters and spaces around the dimensions
    cleaned_dimensions = re.sub(r'[^\d. x]', '', size)
    return cleaned_dimensions

# Calculate the area of each room (where each dict in the "Room Info" list represents a different room)

def get_area(room):
    """House Sigma dfs"""
    # Sometimes the "Room Info" list of dictionaries is an empty list [], or there's no key named "size"
    # for certain rooms in the list.
    if room['size'] is None:
        return 0
    
    # Clean to remove the mï¼‰ special characters after the dimensions for each room
    cleaned_size = clean_size_string(room['size'])
    
    # Use regex to identify form and pattern of dimensions in each dictionary in the list of rooms
    # Provided dimensions are actually in meters.
    match = re.match(r'([\d.]+) x ([\d.]+)', cleaned_size)
    if match:
        length = float(match.group(1))  
        width = float(match.group(2))   
        area_m2 = length * width       
        area_ft2 = area_m2 * 10.7639    
        return area_ft2
    else:
        return 0  
    
# Calculate total sqft (sum of the areas of all rooms in the list of dictionaries)

def get_total_sqft(rooms_list):
    return sum(get_area(room) for room in rooms_list)

In [174]:
def create_columns(list_of_dfs):
    new_dfs = []
    for df in list_of_dfs:
        first_row_empty = df.iloc[0].isna().all()
        
        if first_row_empty:
            df.columns = ['Name', 'Property Info', 'Listing Info', 'Room Info', 'description 1', 'description 2', 'link']
            new_dfs.append(df)
        else:
            new_row = pd.DataFrame([df.columns], columns=df.columns)
            df = pd.concat([new_row, df], ignore_index=True)
            df.columns = ['Name', 'Property Info', 'Listing Info', 'Room Info', 'description 1', 'description 2', 'link']
            new_dfs.append(df)
            
    return new_dfs

In [175]:
def clean_real_estate(new_house_sigma, house_sigma_overview):
    
    """Cleans and processes house_sigma dataframes."""
    
    
    ##### House Sigma #####
        
    # new_house_sigma["Property Info"] = new_house_sigma["Property Info"].apply(ast.literal_eval)
    # new_house_sigma["Listing Info"] = new_house_sigma["Listing Info"].apply(ast.literal_eval)
    # new_house_sigma["Room Info"] = new_house_sigma["Room Info"].apply(ast.literal_eval)

    new_house_sigma["Property Info"] = new_house_sigma["Property Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    new_house_sigma["Listing Info"] = new_house_sigma["Listing Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    new_house_sigma["Room Info"] = new_house_sigma["Room Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    
    property_expanded = pd.json_normalize(new_house_sigma['Property Info'])
    listing_expanded = pd.json_normalize(new_house_sigma['Listing Info'])

    # Concatenate the expanded DataFrames with the original DataFrame, excluding the old columns
    df_expanded = pd.concat([new_house_sigma.drop(columns=['Property Info', 'Listing Info']), property_expanded, listing_expanded], axis=1)
    
    df_expanded['calculated_sqft'] = df_expanded['Room Info'].apply(get_total_sqft)


    # Drop the first instance of each duplicate column (property info and listing info had lots of duplicate keys)
    df_expanded = df_expanded.loc[:, ~df_expanded.columns.duplicated(keep='last')]
    
    df_expanded["Size:"] = df_expanded["Size:"].apply(get_realtor_house_sigma_sqft)

    df_expanded["Size:"] = df_expanded["Size:"].fillna(0)
    df_expanded["sqft"] = df_expanded.apply(lambda row: row["calculated_sqft"] if row["Size:"] == 0 else row["Size:"], axis=1)
    df_expanded = df_expanded.loc[df_expanded['sqft'] != 0]

    house_sigma = df_expanded

    house_sigma["mls"] = house_sigma["Listing #:"]
    house_sigma["beds"] = house_sigma["Bedrooms:"]
    house_sigma["baths"] = house_sigma["Bathrooms:"]
    house_sigma["Heating Type"] = house_sigma["Heating Type:"]
    house_sigma["Air Conditioning"] = house_sigma["Cooling:"]
    house_sigma["Community"] = house_sigma["Community:"]
    
    house_sigma["Frontage"] = house_sigma["Frontage:"].apply(numeric_lot_dim)
    house_sigma["Land Depth"] = house_sigma["Depth:"].apply(numeric_lot_dim)
    
    # house_sigma["Amenities:"] = house_sigma["Amenities:"].str.strip().replace({
    #     "Outdoor Pool": "Pool", 
    #     "Indoor Pool": "Pool"
    # })
    
    # house_sigma["Amenities:"] = house_sigma["Amenities:"].apply(
    #     lambda x: x.replace("Outdoor Pool", "Pool").replace("Indoor Pool", "Pool") if isinstance(x, str) else x
    # )

    # house_sigma['Amenities:'] = house_sigma.apply(
    #     lambda row: (
    #         (str(row['Amenities:']) + ", Pool") if (
    #             pd.notna(row['Pool:']) or
    #             (isinstance(row['description 1'], str) and 'pool' in row['description 1'].lower()) or
    #             (isinstance(row['description 2'], str) and 'pool' in row['description 2'].lower())
    #         ) else row['Amenities:']
    #     ),
    #     axis=1
    # )
    
    house_sigma['Amenities:'] = house_sigma.apply(
        lambda row: (
            (str(row['Amenities:']) + ", Pool") if (
                (pd.notna(row['Pool:']) or
                (isinstance(row['description 1'], str) and 'pool' in row['description 1'].lower()) or
                (isinstance(row['description 2'], str) and 'pool' in row['description 2'].lower())) and
                'pool' not in str(row['Amenities:']).lower()  # Check if "pool" is already in Amenities
            ) else row['Amenities:']
        ),
        axis=1
    )
    
    
    # house_sigma['Amenities:'] = house_sigma.apply(
    #     lambda row: (str(row['Amenities:']) + ", Pool") if (
    #     pd.notna(row['Pool:']) or
    #     (isinstance(row['description 1'], str) and 'pool' in row['description 1'].lower()) or # descriptions not str means that they are NaN or [] --> can ignore without removing these rows
    #     (isinstance(row['description 2'], str) and 'pool' in row['description 2'].lower())
    #     ) else row['Amenities:'],
    #     axis=1
    # )

    # house_sigma['Amenities:'] = house_sigma.apply(
    #     lambda row: (row['Amenities:'] + ", Pool") if (
    #                                                     pd.notna(row['Pool:']) or
    #                                                     row['description 1'].str.contains("pool", case=False, na=False) or
    #                                                     row['description 2'].str.contains("pool", case=False, na=False)
    #                                                 ) else row['Amenities:'],
    #                                                         axis=1
    # )
    
    # house_sigma['Amenities:'] = house_sigma.apply(
    #     lambda row: (row['Amenities:'] + ", Pool") if (pd.notna(row['Pool:']), |
    #                                                     house_sigma["description 1"].str.contains("pool", case=False), |
    #                                                     house_sigma["description 2"].str.contains("pool", case=False)
    #                                                     ) else row['Amenities:'],
    #                                                     axis=1)
    
    # Replace NaN values explicitly with "No Amenity"
    house_sigma["Amenities:"] = house_sigma["Amenities:"].fillna("No amenity")

    house_sigma_merged = pd.merge(house_sigma_overview, house_sigma, on="link", how="left")
    
    # Filter out listings for rent and convert price to float.
    
    house_sigma_merged = house_sigma_merged[~house_sigma_merged["strikethrough_price"].str.contains("Monthly | Weekly", case=False, na=False)]
    house_sigma_merged = numeric_price(house_sigma_merged, ["strikethrough_price", "sold_price"])
    house_sigma_merged["price"] = house_sigma_merged["strikethrough_price"]
    
    house_sigma_merged = house_sigma_merged.rename(columns={"Property Type:" : "Building Type"})
    
    return house_sigma_merged

In [176]:
# def get_top_amenities(realtor, zolo, house_sigma):
#     """Gets bools for top 10 amenities across all three real estate dfs."""
    
#     # Concatenate the three dfs
#     concatenated = pd.concat([realtor, zolo, house_sigma], ignore_index=True)

#     # Function to handle both strings and lists
#     def split_string_or_list(value):
#         # If the value is a string and contains commas, split it
#         if isinstance(value, str):
#             return value.split(',')  # Split by commas to create a list
#         elif isinstance(value, list):
#             return value  # Return the list as is
#         return []  # In case of other types (e.g., NaN or unexpected values)

#     # Exploding the amenities columns, one by one, and handling both strings and lists
#     features_exploded = concatenated['Features'].apply(split_string_or_list).explode().dropna()
#     building_amenities_exploded = concatenated['Building Amenities'].apply(split_string_or_list).explode().dropna()
#     amenity_exploded = concatenated['Amenity'].apply(split_string_or_list).explode().dropna()
#     amenities_exploded = concatenated['Amenities:'].apply(split_string_or_list).explode().dropna()

#     # Concatenate all the exploded lists into a single Series
#     all_amenities = pd.concat([features_exploded, building_amenities_exploded, amenity_exploded, amenities_exploded])

#     # Count the occurrences of each amenity
#     counter = Counter(all_amenities)
    
#     return counter

In [177]:
# def split_string_or_list(value):
#     """Handles both strings and lists, ensuring proper splitting and cleaning."""
#     # If it's a string, ensure proper trimming and splitting
#     if isinstance(value, str):
#         # Convert to lowercase and strip leading/trailing spaces, then split by commas
#         value = value.lower().strip()
#         return value.split(',')  # Split by commas to create a list
#     elif isinstance(value, list):
#         return value  # Return the list as is
#     return []  # In case of NaN or other unexpected types

# def split_string_or_list(value):
#     """Handles both strings and lists, ensuring proper splitting and cleaning."""
#     # If it's a string, ensure proper trimming and splitting
#     if isinstance(value, str):
#         # Convert to lowercase and strip leading/trailing spaces, then split by commas
#         value = value.lower().strip()
#         return [v.strip() for v in value.split(',')]  # Split by commas and strip each item
#     elif isinstance(value, list):
#         return value  # Return the list as is
#     return []


# def get_top_amenities(realtor, zolo, house_sigma):
#     """Gets bools for top amenities across all real estate dfs."""
    
#     # Concatenate the three dfs
#     concatenated = pd.concat([realtor, zolo, house_sigma], ignore_index=True)

#     # Exploding the amenities columns, one by one, and handling both strings and lists
#     features_exploded = concatenated['Features'].apply(split_string_or_list).explode().dropna()
#     building_amenities_exploded = concatenated['Building Amenities'].apply(split_string_or_list).explode().dropna()
#     amenity_exploded = concatenated['Amenity'].apply(split_string_or_list).explode().dropna()
#     amenities_exploded = concatenated['Amenities:'].apply(split_string_or_list).explode().dropna()

#     # Concatenate all the exploded lists into a single Series
#     all_amenities = pd.concat([features_exploded, building_amenities_exploded, amenity_exploded, amenities_exploded])
#     print(all_amenities.unique())
#     # Count the occurrences of each amenity (case-insensitive)
#     counter = Counter(all_amenities)

#     # Print the top 20 and their respective counts.
#     print(counter.most_common(20))

#     return counter

In [178]:
def split_string_or_list(value):
    """Splits a string into a list or returns the value if it's already a list."""
    if isinstance(value, str):
        # If it's a string, split it by commas and strip whitespace
        return [item.strip() for item in value.split(',')]
    elif isinstance(value, list):
        # If it's already a list, return it as-is
        return value
    else:
        # Handle any other cases (e.g., NaN, None)
        return []

def lump_amenities(amenity):
    """Lumps related amenities into broader categories."""
    # Define groups of related amenities (e.g., pool-related, parking-related)
    amenity_groups = {
        'gym': ['gym', 'exercise room', 'exercise centre'],
        'parking': ['visitor parking', 'paved driveway', 'parking'],
        'balcony': ['balcony', 'rooftop deck/garden'],
        'security': ['security/concierge', 'concierge', 'security'],
        'guest suites': ['guest suite', 'guest suites'],
        'party room': ['party room', 'party/meeting room'],
        'fireplace': ['fireplace(s)'],
        'recreation': ['recreation room', 'recreation centre', 'games room'],
        'green_belt':["conservation/green belt", "'backs on greenbelt"],
        'wheelchair access': ["wheelchair access", "level lot", 'level', 'flat site'],
        'no amenity': ["no amenity", "nan", ""]
    }
    amenity_normalized = amenity.strip().lower()
    
    # Loop through the groups and return the group name if amenity matches
    for group, amenities in amenity_groups.items():

        if amenity_normalized in [a.strip().lower() for a in amenities]:
            return group
        
    return amenity  # If no match, return the original amenity (for ungrouped amenities)

def get_top_amenities(house_sigma):
    """Gets bools for top amenities across all real estate dfs."""
    
    # Concatenate the three DataFrames
    concatenated = pd.concat([house_sigma], ignore_index=True)

    # List of columns that contain amenities in your DataFrames
    amenity_columns = ['Amenities:']
    
    # Exploding the amenities columns, one by one, and handling both strings and lists
    exploded_amenities = []
    for col in amenity_columns:
        # Split and explode the values from each column
        exploded_amenities.append(concatenated[col].apply(split_string_or_list).explode().dropna())
    
    # Concatenate all the exploded lists into a single Series
    all_amenities = pd.concat(exploded_amenities)
    
    # Lump similar amenities into their broader groups
    grouped_amenities = all_amenities.apply(lump_amenities)
    
    # Count the occurrences of each group (case-insensitive)
    counter = Counter(grouped_amenities.str.lower())  # Use lower() to count case-insensitively
    
    # Print the top 20 amenities and their respective counts.
    top_20_amenities = counter.most_common(20)
    print("Top 20 Amenities (Grouped):")
    for amenity, count in top_20_amenities:
        print(f"{amenity}: {count}")
    
    return counter


In [179]:
# def merge_real_estate(realtor, zolo, house_sigma):
#     """Gets bools for top 10 amenities across all three real estate dfs."""
    
#     concatenated = pd.concat([realtor, zolo, house_sigma], ignore_index=True)

#     all_amenities = concatenated['Features'].explode().tolist() \
#         + concatenated['Building Amenities'].explode().tolist() \
#         + concatenated['Amenity'].explode().tolist() \
#         + concatenated['Amenities:'].explode().tolist()
    
#     counter = get_top_amenities(realtor, zolo, house_sigma)
#     top_20_amenities = counter.most_common(20)
    
#     top_20_amenity_names = [amenity[0].strip() for amenity in top_20_amenities]
    
#     # Create a new column for each top amenity: 1 if it's in the row, 0 otherwise
#     for amenity in top_20_amenity_names:
#         concatenated[amenity] = concatenated.apply(lambda row: 1 if amenity in row.values else 0, axis=1)
        
#     print("Columns in concatenated dataframe:", concatenated.columns)
#     print("Top 20 amenity names:", top_20_amenity_names)
#     columns_to_select = [
#         "price", 
#         "address", 
#         "mls", 
#         "sqft", 
#         "Community", 
#         "beds", 
#         "baths", 
#         "Air Conditioning", 
#         "Heating"
#     ] + top_20_amenity_names  

#     selected_data = concatenated[columns_to_select]
    
#     return selected_data


In [180]:
def merge_real_estate(house_sigma):
    """Gets bools for top 10 amenities across all three real estate dfs."""
    
    # Concatenate all the DataFrames
    concatenated = pd.concat([house_sigma], ignore_index=True)

    # Get the top amenities from the concatenated DataFrame
    # all_amenities = concatenated['Features'].explode().tolist() \
    #     + concatenated['Building Amenities'].explode().tolist() \
    #     + concatenated['Amenity'].explode().tolist() \
    #     + concatenated['Amenities:'].explode().tolist()

    # Get the top 20 amenities based on the count
    counter = get_top_amenities(house_sigma)
    top_20_amenities = counter.most_common(21) # Used top 21 because "no amenity" shows up as a frequent "amenity".
    
    # Extract the names of the top 20 amenities
    top_20_amenity_names = [amenity[0].strip() for amenity in top_20_amenities]
    print(top_20_amenity_names)
    # Create a new DataFrame with the boolean columns for each amenity
    amenity_columns = pd.DataFrame()

    for amenity in top_20_amenity_names:
        amenity_columns[amenity] = concatenated.apply(lambda row: 1 if amenity in row.values else 0, axis=1)

    # Concatenate the new boolean columns with the original DataFrame
    concatenated = pd.concat([concatenated, amenity_columns], axis=1)

    # Select the relevant columns
    columns_to_select = [
        "price", 
        "address", 
        "mls", 
        "sqft",
        "Community", 
        "beds", 
        "baths",
        "Building Type",
        "Air Conditioning", 
        "Heating Type",
        "lat",
        "long",
        "Frontage",
        "Land Depth",
    ] + top_20_amenity_names  

    selected_data = concatenated[columns_to_select]
    
    return selected_data


In [181]:
def split_string_or_list(value):
    """Splits a string into a list or returns the value if it's already a list."""
    if isinstance(value, str):
        # If it's a string, split it by commas and strip whitespace
        return [item.strip() for item in value.split(',')]
    elif isinstance(value, list):
        # If it's already a list, return it as-is
        return value
    else:
        # Handle any other cases (e.g., NaN, None)
        return []

def lump_amenities(amenity):
    """Lumps related amenities into broader categories."""
    # Define groups of related amenities (e.g., pool-related, parking-related)
    amenity_groups = {
        'gym': ['gym', 'exercise room', 'exercise centre'],
        'parking': ['visitor parking', 'paved driveway', 'parking'],
        'balcony': ['balcony', 'rooftop deck/garden'],
        'security': ['security/concierge', 'concierge', 'security'],
        'guest suites': ['guest suite', 'guest suites'],
        'party room': ['party room', 'party/meeting room'],
        'fireplace': ['fireplace(s)'],
        'recreation': ['recreation room', 'recreation centre', 'games room'],
        'green_belt':["conservation/green belt", "'backs on greenbelt"],
        'wheelchair access': ["wheelchair access", "level lot", 'level', 'flat site'],
        'no amenity': ["no amenity", "nan", ""]
    }
    amenity_normalized = amenity.strip().lower()
    
    # Loop through the groups and return the group name if amenity matches
    for group, amenities in amenity_groups.items():
        if amenity_normalized in [a.strip().lower() for a in amenities]:
            return group
        
    return amenity  # If no match, return the original amenity (for ungrouped amenities)

def check_amenity_groups(concatenated, amenity_groups):
    """Checks if a listing contains any of the grouped amenities and creates boolean columns for each."""
    
    # Convert the relevant columns to lowercase for case-insensitive comparison
    amenity_columns_list = ['Features', 'Building Amenities', 'Amenity', 'Amenities:']
    concatenated[amenity_columns_list] = concatenated[amenity_columns_list].applymap(
        lambda x: str(x).lower() if pd.notnull(x) else ''
    )
    
    # Create a new DataFrame to store the boolean columns for each group
    amenity_columns = pd.DataFrame()
    
    # For each group in the amenity groups, check if any of the values in the relevant columns match
    for amenity_group in amenity_groups:
        # Check if any value in the relevant columns belongs to the current amenity group
        amenity_columns[amenity_group] = concatenated.apply(
            lambda row: 1 if any(
                lump_amenities(str(value)).lower() == amenity_group
                for value in row[amenity_columns_list].dropna().values
            ) else 0,
            axis=1
        )

    # Concatenate the new boolean columns with the original DataFrame
    concatenated = pd.concat([concatenated, amenity_columns], axis=1)

    return concatenated

def merge_real_estate(house_sigma):
    """Merges data and adds boolean columns for top amenities."""
    
    # Concatenate all the DataFrames
    concatenated = pd.concat([house_sigma], ignore_index=True)
    concatenated = concatenated.drop_duplicates(subset=["mls"])
    
    # Get the top amenities from the concatenated DataFrame
    counter = get_top_amenities(house_sigma)
    top_20_amenities = counter.most_common(21)  # Used top 21 because "no amenity" shows up as a frequent "amenity".
    
    # Extract the names of the top amenities
    top_20_amenity_names = [amenity[0].strip() for amenity in top_20_amenities]
    
    # Create a new DataFrame with the boolean columns for each amenity
    amenity_columns = pd.DataFrame()

    for amenity in top_20_amenity_names:
        amenity_lower = amenity.lower()

        amenity_columns[amenity] = concatenated.apply(
            lambda row: 1 if any(amenity_lower in str(value).lower() for value in set(row[['Amenities:']].values)) else 0, axis=1
        )
        
    # Concatenate the new boolean columns with the original DataFrame
    concatenated = pd.concat([concatenated, amenity_columns], axis=1)

    # Select the relevant columns
    columns_to_select = [
        "price", 
        "address", 
        "mls", 
        "sqft",
        "Community", 
        "beds", 
        "baths",
        "Building Type",
        "Air Conditioning", 
        "Heating Type",
        "lat",
        "long",
        "Frontage",
        "Land Depth",
    ] + top_20_amenity_names  

    selected_data = concatenated[columns_to_select]
    
    return selected_data

In [182]:
def final_cleaning(merged):
    """Remove any remaining NaN values by price, address, MLS, and sqft.
    Drop duplicates by MLS across the three real estate data sources."""
    
    merged = merged.dropna(subset=['price', 'address', 'mls', 'sqft', 'lat', 'long'])
                    
    merged = merged.drop_duplicates(subset=["mls"])
    
    return merged

In [183]:
# Listings through the years

parent_dir = Path.cwd().parent
hs0 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 0.csv")
hs2 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 2.csv")
hs3 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 3.csv")
hs4 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 4 - Copy.csv") #, encoding="utf-8", on_bad_lines='skip')
hs5 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 5.csv")

hs_overview = pd.read_csv(parent_dir/"2_data_cleaning"/"cleaned_csv"/"housesigma_data_2003_with_coords_date.csv")

In [184]:
dfs_list = create_columns([hs0, hs2, hs3, hs4, hs5])
hs_total = pd.concat(dfs_list, ignore_index=True)
hs_total.shape

(87995, 7)

In [185]:
house_sigma_cleaned = clean_real_estate(hs_total, hs_overview)
merged = merge_real_estate(house_sigma_cleaned)

merged.isna().sum()

Top 20 Amenities (Grouped):
no amenity: 112825
pool: 9163
bbqs allowed: 4978
parking: 4555
bike storage: 1496
gym: 1376
party room: 1359
balcony: 1029
security system: 781
security: 548
indoor pool: 540
security guard: 454
outdoor pool: 427
recreation: 393
sauna: 300
guest suites: 280
car wash: 151
media room: 76
tennis court: 75
squash/racquet court: 40


price                    152
address                    0
mls                        1
sqft                       1
Community                  1
beds                       5
baths                      3
Building Type              1
Air Conditioning        9361
Heating Type              16
lat                       61
long                      61
Frontage                6256
Land Depth              6256
no amenity                 0
pool                       0
bbqs allowed               0
parking                    0
bike storage               0
gym                        0
party room                 0
balcony                    0
security system            0
security                   0
indoor pool                0
security guard             0
outdoor pool               0
recreation                 0
sauna                      0
guest suites               0
car wash                   0
media room                 0
tennis court               0
squash/racquet court       0
bus ctr (wifi 

In [186]:
final_merged = final_cleaning(merged)

In [187]:
final_merged["price"].isna().sum()

np.int64(0)

In [188]:
final_merged[final_merged["pool"] == 1]

,price,address,mls,sqft,Community,beds,baths,Building Type,Air Conditioning,Heating Type,...,security guard,outdoor pool,recreation,sauna,guest suites,car wash,media room,tennis court,squash/racquet court,bus ctr (wifi bldg)
28,939000.0,"195 phyllis ave , scarborough - cliffcrest",E10432118,1023.274459,Cliffcrest,3,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
91,1269900.0,"112 toynbee tr , scarborough - guildwood",E9308174,1553.640875,Guildwood,4,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
93,5380000.0,"19 hedgewood rd , north york - bridle path-sun...",C9768843,1859.609038,Bridle Path-Sunnybrook-York Mills,3,5,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
106,1099000.0,"18 gillespie ave , toronto - weston-pellam park",W10432746,1750.000000,Weston-Pellam Park,3,3,Semi-Detached,Wall Unit,Radiant,...,0,0,0,0,0,0,0,0,0,0
107,999880.0,"149 avondale ave , north york - willowdale east",C9282340,617.710082,Willowdale East,3,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123919,309900.0,"(address not available) , scarborough - eglint...",E1130464,1505.975096,Eglinton East,4,2,Detached,NaN,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123946,259900.0,"(address not available) , toronto - corso ital...",W8293732,1750.000000,Corso Italia-Davenport,3,4,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123971,199950.0,"(address not available) , toronto - caledonia-...",W5932124,1485.132957,Caledonia-Fairbank,3,4,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123979,3200000.0,"(address not available) , north york - bridle ...",C3069753,5000.000000,Bridle Path-Sunnybrook-York Mills,4,7,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0


In [189]:
final_merged.shape

(65473, 35)

In [190]:
final_merged.to_csv("house_sigma_historical_data_nov30.csv")

In [191]:
final_merged.isna().sum()

price                      0
address                    0
mls                        0
sqft                       0
Community                  0
beds                       4
baths                      2
Building Type              0
Air Conditioning        9320
Heating Type              15
lat                        0
long                       0
Frontage                6223
Land Depth              6222
no amenity                 0
pool                       0
bbqs allowed               0
parking                    0
bike storage               0
gym                        0
party room                 0
balcony                    0
security system            0
security                   0
indoor pool                0
security guard             0
outdoor pool               0
recreation                 0
sauna                      0
guest suites               0
car wash                   0
media room                 0
tennis court               0
squash/racquet court       0
bus ctr (wifi 

In [192]:
merged[merged["pool"]==1]

,price,address,mls,sqft,Community,beds,baths,Building Type,Air Conditioning,Heating Type,...,security guard,outdoor pool,recreation,sauna,guest suites,car wash,media room,tennis court,squash/racquet court,bus ctr (wifi bldg)
28,939000.0,"195 phyllis ave , scarborough - cliffcrest",E10432118,1023.274459,Cliffcrest,3,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
91,1269900.0,"112 toynbee tr , scarborough - guildwood",E9308174,1553.640875,Guildwood,4,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
93,5380000.0,"19 hedgewood rd , north york - bridle path-sun...",C9768843,1859.609038,Bridle Path-Sunnybrook-York Mills,3,5,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
106,1099000.0,"18 gillespie ave , toronto - weston-pellam park",W10432746,1750.000000,Weston-Pellam Park,3,3,Semi-Detached,Wall Unit,Radiant,...,0,0,0,0,0,0,0,0,0,0
107,999880.0,"149 avondale ave , north york - willowdale east",C9282340,617.710082,Willowdale East,3,2,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123919,309900.0,"(address not available) , scarborough - eglint...",E1130464,1505.975096,Eglinton East,4,2,Detached,NaN,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123946,259900.0,"(address not available) , toronto - corso ital...",W8293732,1750.000000,Corso Italia-Davenport,3,4,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123971,199950.0,"(address not available) , toronto - caledonia-...",W5932124,1485.132957,Caledonia-Fairbank,3,4,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0
123979,3200000.0,"(address not available) , north york - bridle ...",C3069753,5000.000000,Bridle Path-Sunnybrook-York Mills,4,7,Detached,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,0


In [193]:
final_merged[final_merged["sauna"]==1]

,price,address,mls,sqft,Community,beds,baths,Building Type,Air Conditioning,Heating Type,...,security guard,outdoor pool,recreation,sauna,guest suites,car wash,media room,tennis court,squash/racquet court,bus ctr (wifi bldg)
951,709000.0,"603 - 8 widmer st , toronto - waterfront commu...",C9367376,749.500000,Waterfront Communities C1,2,2,Condo Townhouse,Central Air,Forced Air,...,0,1,0,1,0,0,0,0,0,0
1201,1198800.0,"118 - 208 niagara st , toronto - niagara",C9410045,1099.500000,Niagara,2,2,Condo Townhouse,Central Air,Forced Air,...,0,0,0,1,0,0,0,0,0,0
2023,620000.0,"207 - 40 sunny glwy , toronto - flemingdon park",C9245333,1099.500000,Flemingdon Park,4,2,Condo Townhouse,Other,Baseboard,...,0,0,1,1,0,0,0,0,0,0
2891,599800.0,"3 - 91 muir dr , scarborough - scarborough vil...",E9035010,1299.500000,Scarborough Village,3,2,Condo Townhouse,Wall Unit,Baseboard,...,0,0,1,1,0,0,0,0,0,0
3513,649000.0,"19 - 208 niagara st , toronto - niagara",C9044655,749.500000,Niagara,1,1,Condo Townhouse,Central Air,Forced Air,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121583,149000.0,"215 - 95 leeward glenway , toronto - flemingdo...",C334042,1082.316603,Flemingdon Park,3,2,Condo Townhouse,NaN,Baseboard,...,1,0,1,1,0,0,0,0,0,0
121840,139900.0,"32 - 91 muir dr , scarborough - scarborough vi...",E296301,817.485913,Scarborough Village,3,2,Condo Townhouse,NaN,Baseboard,...,0,0,1,1,0,0,0,0,0,0
122310,192000.0,"(address not available) , toronto - niagara",C296474,636.798782,Niagara,1,1,Condo Townhouse,Central Air,Forced Air,...,0,0,1,1,0,0,0,0,0,0
122426,146900.0,"(address not available) , scarborough - scarbo...",E5650722,1299.500000,Scarborough Village,3,2,Condo Townhouse,Other,Baseboard,...,0,0,1,1,0,0,0,0,0,0
